<a href="https://colab.research.google.com/github/mjaffery9076/mjaffery9076/blob/main/AI_Projecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy
import scrapy
from scrapy import Request
from scrapy.selector import Selector
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [13]:

# Define the model architecture
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(784,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model with an optimizer and loss function
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Load and preprocess the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((60000, 784)) / 255.0
x_test = x_test.reshape((10000, 784)) / 255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Train the model for a specified number of epochs
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

# Print the test accuracy
print('Test accuracy:', test_acc)


Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: -4968526.0000 - accuracy: 0.1123 - val_loss: -19300132.0000 - val_accuracy: 0.1135
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: -59507956.0000 - accuracy: 0.1124 - val_loss: -118060560.0000 - val_accuracy: 0.1135
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: -206502384.0000 - accuracy: 0.1124 - val_loss: -322282240.0000 - val_accuracy: 0.1135
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: -466511840.0000 - accuracy: 0.1124 - val_loss: -651759744.0000 - val_accuracy: 0.1135
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: -858863232.0000 - accuracy: 0.1124 - val_loss: -1126316032.0000 - val_accuracy: 0.1135
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: -1405040000.0000 - accuracy: 0.1124 - val_loss: -1768724736.0000 - val_accuracy: 0.1135
Epoch 7/10
1875/1875 [=======

In [14]:
import tensorflow as tf

# Define the training data
train_data = [
    ('I love this sandwich', 1),
    ('This is an amazing place', 1),
    ('I feel very good about these beers', 1),
    ('This is my best work', 1),
    ("What an awesome view", 1),
    ('I do not like this restaurant', 0),
    ('I am tired of this stuff', 0),
    ("I can't deal with this", 0),
    ('He is my sworn enemy', 0),
    ('My boss is horrible', 0)
]

# Define the vocabulary size and the maximum length of input sequences
vocab_size = 50
max_length = 10

# Define the tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts([x[0] for x in train_data])

# Convert the training data into sequences
train_sequences = tokenizer.texts_to_sequences([x[0] for x in train_data])
train_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_padded_sequences, [x[1] for x in train_data], epochs=20, batch_size=1)

# Save the model
model.save('AIProject.h5')


ValueError: ignored

In [7]:


class TwitterSpider(scrapy.Spider):
    name = "twitter"

    def __init__(self, query="", **kwargs):
        self.allowed_domains = ["twitter.com"]
        self.start_urls = [f"https://twitter.com/search?q={query}&src=typed_query"]
        super().__init__(**kwargs)

    def start_requests(self):
        for url in self.start_urls:
            yield Request(url, callback=self.parse)

    def parse(self, response):
        # Extract the text of each ping using a CSS selector
        ping_text = response.css("div.tweet-text::text").getall()

        # Evaluate the sentiment of each ping using the Tensorflow model
        positive_words = [('nice', 0.8), ('good', 0.9), ('cool', 0.7)]
        negative_words = [('bad', 0.8), ('horrible', 0.9)]
        model = tf.keras.models.load_model('sentiment_analysis_model.h5')
        positive_pings = []
        negative_pings = []
        for text in ping_text:
            encoding = preprocess_data(text)
            sentiment = model.predict(encoding)[0][0]
            if sentiment > 0.5:
                positive_pings.append(text)
            else:
                negative_pings.append(text)

        # Output the positive and negative pings
        print("Positive pings:")
        for ping in positive_pings:
            print(ping)
        print("Negative pings:")
        for ping in negative_pings:
            print(ping)

def preprocess_data(text):
    # Tokenize the text
    tokens = text.lower().split()

    # Create a weighted one-hot encoding of the text
    positive_words = [('nice', 0.8), ('good', 0.9), ('cool', 0.7)]
    negative_words = [('bad', 0.8), ('horrible', 0.9)]
    one_hot_encoding = [word_weight * int(word == token) for token in tokens for word, word_weight in (positive_words + negative_words)]

    # Reshape the encoding to match the input shape of the model
    encoding = np.reshape(one_hot_encoding, (1, len(one_hot_encoding)))

    return encoding
